In [1]:

# from duong.gen_angelic_st import GenAngelic
from minh.deepseek import chat



# chat = GenAngelic().init()
class Answerer:
    def __init__(self, debug=False):
        self.debug = debug

    def answer(self, article, query):
        reply = chat(article, query)
        if self.debug:
            print("======== Generate Angelic structure and Query Entailment: ========\n", reply)

text = ""
with open("./test.txt", "r") as f:
    text = "".join(f.readlines())
answerer = Answerer(debug=True)
answer = answerer.answer(text, "In cases where an individual rescues another person from getting hit by a car by pushing that person out of the way, causing the person's luxury kimono to get dirty, the rescuer does not have to compensate damages for the kimono.")
print(answer)


======== Generate Angelic structure and Query Entailment: ========
 To construct the ANGELIC structure based on the provided legal provisions and then analyze the given query, we follow a step-by-step approach.

### Step 1: Identify the Root Node (R)
The article seeks to establish that if a manager engages in benevolent intervention to help another person avoid imminent danger without acting in bad faith or with gross negligence, the manager is not liable for compensation for any resulting damage. Thus, the root node can be defined as:
- **R1**: A manager is not liable for damages when intervening benevolently without bad faith or gross negligence.

### Step 2: Derive Abstract Factors (A)
The key legal factors that validate this conclusion include:
- **A1**: The intervention was made to prevent imminent danger.
- **A2**: The intervention was done in good faith and without gross negligence.

### Step 3: Refine Abstract Factors into Base-Level Factors (B), if applicable
Refining the abst

In [4]:
from extractor.extractor import MDAExtractor
# from duong.gen_angelic_st import GenAngelic
from reasoner.reasoner import Reasoner
import dspy

# chat = GenAngelic().init()
class Answerer:
    def __init__(self, reply, debug=False):
        self.extractor = MDAExtractor()
        self.reasoner = Reasoner()
        self.debug = debug
        self.reply = reply

    def answer(self, article, query):
        facts = self.extractor.forward(text=self.reply)
        if self.debug:
            print("======= Fact Extractor step ========\n")
            print("Facts:\n", facts)
            print("\n")
        # theory_2
        return self.reasoner.reason(facts)



dspy.settings.configure(
    lm=dspy.LM(
        model="ollama_chat/llama3.3",  # Changed to include provider prefix
        api_base="http://localhost:11435",
        max_tokens=20000,
    )
)
text = ""
reply = ""
with open("./test_reply.txt", "r") as f:
    reply = "".join(f.readlines())
answerer = Answerer(reply=reply, debug=True)

answer = answerer.answer(text, "In cases where an individual rescues another person from getting hit by a car by pushing that person out of the way, causing the person's luxury kimono to get dirty, the rescuer does not have to compensate damages for the kimono.")
print(answer)


======= Fact Extractor step ========

Facts:
 link(support(node(afactor(1)),node(root(1)))).link(support(node(afactor(2)),node(root(1)))).link(support(node(bfactor(1)),node(afactor(1)))).link(support(node(bfactor(2)),node(afactor(1)))).link(support(node(bfactor(3)),node(afactor(2)))).entail(query, node(bfactor(2))).entail(query, node(bfactor(3))).agree(query, node(root(1))).


True
